# 1. Install optimization model package

In [ ]:
!pip install -q pyomo
# solvers needed to be installed separately

# glpk
!apt-get install -y -qq glpk-utils

!pip install geopy

!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package


     |████████████████████████████████| 9.7 MB 9.0 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [ ]:
# check for license expiration date
model_size_limited = gp.Model()

Restricted license - for non-production use only - expires 2024-10-28


In [ ]:
import pandas as pd
import numpy as np
from geopy import distance

# 2. Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing
!ls

traffic = pd.read_csv("Data/wa_traffic_counts.csv")[['Y', 'X', 'AADT']].dropna().values.tolist()

main_1 = pd.read_csv('Data/main_dataset_all_interstate_all_route_11_09.csv')
main_1['num_ev_charger_cnt_5mile'] = main_1['num_ev_2_charger_cnt_5mile']+main_1['num_ev_fast_charger_cnt_5mile']
main_1 = main_1.drop(columns = [ 'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit','highway',])
main_1.columns

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Preprocessing
Attractions.ipynb		 I90.ipynb
combine_data.ipynb		 map_request_api.py
Combine.ipynb			 num_EV_chargers_around_gas.ipynb
Data				 OBSOLETE_Geo.ipynb
EV_station_count_distance.ipynb  OBSOLETE_remoteDataList
gas_station_w_traffic.csv	 __pycache__
HighwayExitDistance.ipynb	 traffic_population_crime_combination.ipynb


Index(['Unnamed: 0', 'gas_key', 'gas_name', 'gas_lat', 'gas_long',
       'attr_cnt_1mile', 'attr_cnt_5mile', 'attr_name', 'attr_lat',
       'attr_long', 'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zipcode', 'census_tract_area', 'census_tract_category',
       'traff_cn

In [ ]:
# main_2 = pd.read_csv("Data/main_dataset_all_interstate_all_route_10mi.csv")
main_2 = pd.read_csv("Data/main_dataset_all_interstate_all_route.csv")
main_2 = main_2[['gas_key', 'distance_to_nearest_exit', 'highway']]
# main_2 = main_2.rename(columns = { 'exit': 'highway'})

In [ ]:
main = main_1.merge(main_2, on = 'gas_key')
main.columns

Index(['Unnamed: 0', 'gas_key', 'gas_name', 'gas_lat', 'gas_long',
       'attr_cnt_1mile', 'attr_cnt_5mile', 'attr_name', 'attr_lat',
       'attr_long', 'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zipcode', 'census_tract_area', 'census_tract_category',
       'traff_cn

# 3. Optimization
*Objective function:* Minimize number of chargers/stations

*Constraints:*
- Distance to highway exit < 5 miles
- **All traffic covered (# EV stations * ratio < traffic around).**

Electric vehicles were 1.3% of all passenger vehicles on Washington roads

In [ ]:
df = main[main['highway'].notna()]
df.shape

(1370, 56)

In [ ]:
# df = main[main['highway'] == 'I5']
df_x = df[df['distance_to_nearest_exit'] < 5]
#df_x = df_x[df_x['nri_risk_rating']!= 'Very High']
df_x.shape

(1357, 56)

In [ ]:
def get_adjacenct_gas(candidates, radius = 1):
  # candidates = df_x['gas_key']
  adj = dict()
  for i in range(len(candidates)-1):
      for j in range(i+1, len(candidates)):
        # print(i,j)
        coords_1 = (main.iloc[candidates.iloc[i]]['gas_lat'], main.iloc[candidates.iloc[i]]['gas_long'])
        coords_2 = (main.iloc[candidates.iloc[j]]['gas_lat'], main.iloc[candidates.iloc[j]]['gas_long'])
        dist = distance.distance(coords_1, coords_2).miles
        if dist < radius: 
          if candidates.iloc[i] in adj:
            adj[candidates.iloc[i]].append(candidates.iloc[j])
          else:
            adj[candidates.iloc[i]] = [candidates.iloc[j]]
          if candidates.iloc[j] in adj:
            adj[candidates.iloc[j]].append(candidates.iloc[i])
          else:
            adj[candidates.iloc[j]] = [candidates.iloc[i]]
  for i in range(len(candidates)):
    if candidates.iloc[i] not in adj:
      adj[candidates.iloc[i]] = []

  return adj

# def filter_gas(candidates, radius =1):
#   # remove adjacent gas stations from list, keep only one
#   adj = get_adjacenct_gas(candidates, radius)
#   result = dict()
#   for i in adj.keys():
#     tmp = True
#     if i in result: tmp = False
#     for j in adj[i]:
#       # print(j, j in result)
#       if j in result: tmp = False
#     if tmp: result[i] = len(adj[i])
#   return result


In [ ]:
# adj_sorted = {k: v for k, v in sorted(adj.items(), key=lambda item: len(item[1]))}
# adj_sorted

# filtered_gas = filter_gas(df_x['gas_key'], radius =2.5)

# filtered_df = pd.DataFrame(filtered_gas.values(),index=filtered_gas.keys()).reset_index()
# filtered_df = filtered_df.rename(columns={'index': 'gas_key', 0: 'nearby_gas'})
# df_x_filtered = filtered_df.merge(df_x, how = 'right', right_on = 'gas_key', left_on = 'gas_key').dropna(subset = ['nearby_gas'])
#df_x_filtered = df_x


In [ ]:
cands = df_x['gas_key']
adj = get_adjacenct_gas(cands, 5)

## Start Optimization

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# p = 0.008 # 0.02, 0.04, 0.1
charger_ratio_1mile = 223.57428393576143/0.02*0.016 # # of EV traffic covered for each charger
#charger_ratio_2mile = 63.31042720630594/0.02*0.016
traffic_captured_p = 1 # % of total EV traffic captured

def ev_station_loc_optimize(input, adjacent, p, traffic_captured_p = 1, limit=10, ratio = charger_ratio_1mile, station_cost = 50000, level2_cost = 7000):

  m = gp.Model()

  # variables
  A = input['gas_key'].tolist()
  # number of EVs nearby 5 miles -> dict
  n = input[['gas_key', 'num_ev_charger_cnt_5mile']].set_index('gas_key').T.to_dict("index")['num_ev_charger_cnt_5mile'] # change to charger number
  # traffic count -> dict
  T = input[['gas_key', 'traff_cnt_5m_max']].set_index('gas_key').T.to_dict("index")['traff_cnt_5m_max'] # tried with avg

  # Add gas station locations
  y = m.addVars(A, vtype=GRB.INTEGER, name='charger') 

  # Set objective function
  m.setObjective(y.sum(), GRB.MINIMIZE) #station_cost*x.sum() + 

  # temporary facilities capacity constraints
  # demand_constraints = m.addConstrs((ratio*(n[i]+y[i]) >= T[i]*p for i in A)) #x[i]*
  # charger_constraints = m.addConstrs(y[i]<=limit for i in A)
  demand_constraints = m.addConstrs((ratio*(n[i]+y[i] + gp.quicksum(y[j] for j in adjacent[i])) >= T[i]*p for i in A)) 
  
  m.optimize()

  result = {}
  # print(f"\n_____________Plan for temporary facilities______________________")
  for i in A:
    if (y[i].x > 0):
      print(f"Build an EV station at location {i}, y: {y[i].x}") #, x: {x[i].x}
      result[i] = y[i].x
  return result


In [ ]:
def ev_station_loc_optimize_dist(input, adjacent, p, limit=10, ratio = charger_ratio_1mile, station_cost = 50000, level2_cost = 7000):

  m = gp.Model()

  # variables
  A = input['gas_key'].tolist()
  # number of EVs nearby 5 miles -> dict
  n = input[['gas_key', 'num_ev_charger_cnt_5mile']].set_index('gas_key').T.to_dict("index")['num_ev_charger_cnt_5mile'] # change to charger number
  # traffic count -> dict
  T = input[['gas_key', 'traff_cnt_5m_max']].set_index('gas_key').T.to_dict("index")['traff_cnt_5m_max'] # tried with avg

  # distance to nearest exit
  D = input[['gas_key', 'distance_to_nearest_exit']].set_index('gas_key').T.to_dict("index")['distance_to_nearest_exit'] # tried with avg
  
  #G = input[['gas_key', 'nearby_gas']].set_index('gas_key').T.to_dict("index")['nearby_gas'] # tried with avg

  # Add gas station locations
  #x = m.addVars(A, vtype=GRB.BINARY, name='gas')
  y = m.addVars(A, vtype=GRB.INTEGER, name='charger') 

  # Set objective function  
  m.setObjective(gp.quicksum(D[i] * y[i] for i in A), GRB.MINIMIZE) #station_cost*x.sum() + 

  # temporary facilities capacity constraints
  # demand_constraints = m.addConstrs((ratio*(n[i]+y[i]) >= T[i]*p for i in A)) #x[i]*
  # charger_constraints = m.addConstrs(y[i]<=limit for i in A)
 
  demand_constraints = m.addConstrs((ratio*(n[i]+y[i] + gp.quicksum(y[j] for j in adjacent[i])) >= T[i]*p for i in A))   

  m.optimize()

  result = {}
  # print(f"\n_____________Plan for temporary facilities______________________")
  for i in A:
    if (y[i].x > 0):
      print(f"Build an EV station at location {i}, y: {y[i].x}") #, x: {x[i].x}
      result[i] = y[i].x
  return result


In [ ]:
def ev_station_loc_optimize_cost(input, adjacent, p, limit=10, ratio = charger_ratio_1mile, station_cost = 50000, level2_cost = 7000):

  m = gp.Model()

  # variables
  A = input['gas_key'].tolist()
  # number of EVs nearby 5 miles -> dict
  n = input[['gas_key', 'num_ev_charger_cnt_5mile']].set_index('gas_key').T.to_dict("index")['num_ev_charger_cnt_5mile'] # change to charger number
  # traffic count -> dict
  T = input[['gas_key', 'traff_cnt_5m_max']].set_index('gas_key').T.to_dict("index")['traff_cnt_5m_max'] # tried with avg

  # distance to nearest exit
  D = input[['gas_key', 'distance_to_nearest_exit']].set_index('gas_key').T.to_dict("index")['distance_to_nearest_exit'] # tried with avg
  
  #G = input[['gas_key', 'nearby_gas']].set_index('gas_key').T.to_dict("index")['nearby_gas'] # tried with avg

  # Add gas station locations
  #x = m.addVars(A, vtype=GRB.BINARY, name='gas')
  y = m.addVars(A, vtype=GRB.INTEGER, name='charger') 

  # Set objective function  
  # m.setObjective(gp.quicksum(D[i] * y[i] for i in A), GRB.MINIMIZE) #station_cost*x.sum() + 

  m.setObjective(gp.quicksum(station_cost * (1 if y[i] else 0) + level2_cost * y[i] for i in A), GRB.MINIMIZE)

  # temporary facilities capacity constraints
  # demand_constraints = m.addConstrs((ratio*(n[i]+y[i]) >= T[i]*p for i in A)) #x[i]*
  charger_constraints = m.addConstrs(y[i]<=limit for i in A)
 
  demand_constraints = m.addConstrs((ratio*(n[i]+y[i] + gp.quicksum(y[j] for j in adjacent[i])) >= T[i]*p for i in A))   

  m.optimize()

  result = {}
  # print(f"\n_____________Plan for temporary facilities______________________")
  for i in A:
    if (y[i].x > 0):
      print(f"Build an EV station at location {i}, y: {y[i].x}") #, x: {x[i].x}
      result[i] = y[i].x
  return result


# 4. Plot Results

In [ ]:
import folium
from folium.features import DivIcon
from folium import plugins

# traffic = pd.read_csv("Data/wa_traffic_counts.csv")[['Y', 'X', 'AADT']].dropna().values.tolist()

def plot_result(result, traffic_df=traffic):
  plot_df = pd.DataFrame({'Latitude': pd.Series(dtype='str'),
                         'Longitude': pd.Series(dtype='str'),
                         'type': pd.Series(dtype='str'),
                         'color': pd.Series(dtype='str'),
                         'radius': pd.Series(dtype='str'),
                         'gas_key': pd.Series(dtype='str'),
                         'charger_count': pd.Series(dtype='str')}
                   )
  for i in result.keys():
    new_df = pd.DataFrame([[main.iloc[i]['gas_lat'], main.iloc[i]['gas_long'], 'Proposed', 'blue', 100, i, int(result[i])]], columns=plot_df.columns)
    plot_df = pd.concat([plot_df, new_df], ignore_index=True)
  
  plot_df = plot_df.sort_values(by=['Latitude'])


  ev_station = pd.read_csv('Data/wa_EV_stations.csv')
  ev_station = ev_station[['Latitude','Longitude']]
  ev_station['type'] = 'EV'
  ev_station['color'] = 'gray'
  ev_station['radius'] = 50
  ev_station['gas_key'] = ''
  ev_station['charger_count'] = ''
  plot_df = pd.concat([ev_station, plot_df], ignore_index=True)

  # plot_df = plot_df.sort_values(by=['Latitude'])
  # print(plot_df)
  #center = 47.139007936370724, -122.52432954892501
  center = 47.392613335075914, -120.35623657380664

  m = folium.Map(location=center, 
                zoom_start=7,
                width=800,height=600)

  # Same as before... go through each home in set, make circle, and add to map.
  # This time we add a color using price and the colormap object
  for i in range(plot_df.shape[0]):
      folium.Circle(
          location=[plot_df.iloc[i]['Latitude'], plot_df.iloc[i]['Longitude']],
          popup=plot_df.iloc[i]['gas_key'],
          radius=1500,
          fill=True,
          color = plot_df.iloc[i]['color'],
          fill_opacity=0.2
      ).add_to(m)

  for i in range(plot_df.shape[0]):
    if i%2==0:
      folium.map.Marker([plot_df.iloc[i]['Latitude'], plot_df.iloc[i]['Longitude']],
                      icon=DivIcon(
                          icon_size=(-10,15),
                          icon_anchor=(-10,14),
                          html=f'<div style="font-size: 14pt">%s</div>' % plot_df.iloc[i]['charger_count'],
                      )
                     ).add_to(m)
    else:
      folium.map.Marker([plot_df.iloc[i]['Latitude'], plot_df.iloc[i]['Longitude']],
                      icon=DivIcon(
                          icon_size=(-10,15),
                          icon_anchor=(25,14),
                          html=f'<div style="font-size: 14pt">%s</div>' % plot_df.iloc[i]['charger_count'],
                      )
                     ).add_to(m)

   # add traffic count heatmap
  plugins.HeatMap(traffic_df,radius=15).add_to(m)

  return m


### 2022: p=1.6%

In [ ]:
result_dict = ev_station_loc_optimize_dist(df_x, adj, p=0.016)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

In [ ]:
result_dict = ev_station_loc_optimize(df_x, adj, p=0.016)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

Build an EV station at location 116, y: 4.0
Build an EV station at location 429, y: 4.0
Build an EV station at location 625, y: 6.0
Build an EV station at location 1048, y: 5.0
Build an EV station at location 1100, y: 7.0
Build an EV station at location 1308, y: 6.0
Build an EV station at location 1652, y: 4.0
Build an EV station at location 1764, y: 5.0


In [ ]:
df_result = pd.DataFrame.from_dict(result_dict, orient ='index').reset_index()
df_result.columns = ['gas_key', 'proposed_chargers']

%cd /content/drive/My Drive/Capstone-KPMG/Modeling
df_result.to_csv('optimization_result.csv', index = False)

### 2026: 3%

In [ ]:
result_dict = ev_station_loc_optimize_dist(df_x, adj, p=0.03)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

In [ ]:
result_dict = ev_station_loc_optimize(df_x, adj, p=0.03)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

### 2030: 5%

In [ ]:
result_dict = ev_station_loc_optimize_dist(df_x, adj, p=0.05)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

In [ ]:
result_dict = ev_station_loc_optimize(df_x, adj, p=0.05)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

In [ ]:
print(pd.__version__)
print(np.__version__)
print(folium.__version__)
print(gp.gurobi.version())
import geopy as geo
print(geo.__version__)
